# Using Binary White Dwarfs as a Gravitational Wave Detector

In [ ]:
import numpy as np

import scipy.constants as scc
from scipy.fft import fft, ifft
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

from plots import plotTF

%config InlineBackend.figure_format = 'retina'
# plt.rc('text', usetex=True) # Google co-lab doesn't have LaTeX!
plt.rc('font', family='serif')

Here we explore the possibility of using gravitational waves (GWs) emitted by binary white dwarfs as a GW detector, similar to the idea of using radio pulses from pulsars to detect very-low frequency (VLF) GWs. First we will give a very brief introduction to the use of pulsar timing array (PTA) for the detection of GWs, and in the second part we will try to try some analogies from PTA to use LF GWs to detect VLF GWs.

## PTA: a crash course

Pulsars are constantly emitting radio pulses. When the beam of radio waves crosses the line of sight between the Earth and a pulsar, a radio pulse will be registered by radio telescopes on Earth (you can think of pulsars as lighthouses, so to speak). For more details at an accessible level, see [this presentation here](http://ipta.phys.wvu.edu/files/student-week-2014/rlynch_ipta2014_student_week_intro_to_ptas.pdf); or if you are a LIGO/Virgo (not talking about the constellations) person, you can also read [this paper](https://journals.aps.org/prd/pdf/10.1103/PhysRevD.79.084030) written by people you should have heard of. When a GW is propagating through the pulsars and the Earth, the light travel time between the pulsars and the Earth will be modified. Suppose a pulsar is located at a direction $\hat{p}$, and that a GW is propagating towards the Earth along a direction $\hat{\Omega}$, then the 'redshift' of the pulse repetition rate frequency $\nu_{\rm em}$ at the source frame is given by

$$ \frac{\nu_{\rm em} - \nu_{\rm obs}}{\nu_{\rm em}} = \frac{1}{2} \frac{\hat{p}^i \hat{p}^j}{1 + \hat{p} \cdot \hat{\Omega}} \left( h_{ij}^{P} - h_{ij}^{E} \right) = G \left( h_{ij}^{P} - h_{ij}^{E} \right) $$

where $G$ is some geometrical factor depending on the alignment of the Earth and the pulsar of interest. The `Earth-term' (actually **at the solar barycenter**, since luckily the Earth is orbiting the Sun) $h_{ij}^{E}$ can be written as
$$ h_{ij}^{E} = h_{ij}(t_{E}, \hat{\Omega}) = \sum_{{\rm pol} = +,\times} h_{{\rm pol}}(t_{E} - \hat{\Omega}\cdot \vec{x}_{E}; \vec{0}) e^{{\rm pol}}_{ij}(\hat{\Omega}) = \sum_{{\rm pol} = +,\times} \int_{-\infty}^{+\infty} df \; e^{2\pi i f (t_{E} - \hat{\Omega}\cdot \vec{x}_{E})} \tilde{h}_{{\rm pol}}(f, \hat{\Omega}) e^{{\rm pol}}_{ij}(\hat{\Omega})$$

and similarly for the `pulsar term' $h_{ij}^{P}$. Therefore we can write the redshift expression as
$$ \frac{\nu_{\rm em} - \nu_{\rm obs}}{\nu_{\rm em}} = \frac{1}{2} \frac{\hat{p}^i \hat{p}^j}{1 + \hat{p} \cdot \hat{\Omega}}  \sum_{{\rm pol} = +,\times} e^{{\rm pol}}_{ij}(\hat{\Omega}) \left[ h_{{\rm pol}}(t_{E} - \hat{\Omega}\cdot \vec{x}_{E}; \vec{0}) - h_{{\rm pol}}(t_{P} - \hat{\Omega}\cdot \vec{x}_{P}; \vec{0}) \right] $$

Suppose we adopted a particular coordinate system where the solar barycenter is at the coordinate origin, and putting the pulsar of interest at a location $\vec{x}_{P} = L \hat{p}$. If we denote $t \equiv t_{E}$, and note that $t_{P} = t - L$, then we can write the redshift formula in *frequency domain* as
$$ \widetilde{\frac{\nu_{\rm em} - \nu_{\rm obs}}{\nu_{\rm em}} }(f) \; = \sum_{{\rm pol} = +,\times} \frac{1}{2} \frac{\hat{p}^i \hat{p}^j}{1 + \hat{p} \cdot \hat{\Omega}} e^{{\rm pol}}_{ij}(\hat{\Omega}) \left( e^{-2\pi ifL(1 + \hat{p} \cdot \hat{\Omega})} - 1\right) \tilde{h}_{{\rm pol}}(f, \hat{\Omega}) $$
where you can think of the term $F_{{\rm pol}} (f) \equiv \frac{1}{2} \frac{\hat{p}^i \hat{p}^j}{1 + \hat{p} \cdot \hat{\Omega}} e^{{\rm pol}}_{ij}(\hat{\Omega}) \left( e^{-2\pi ifL(1 + \hat{p} \cdot \hat{\Omega})} - 1\right)$ as the frequency-dependent GW response function.


Since contribution from this term will add up coherently (i.e. the $-1$ contribution in the expression above) when we consider data from multiple pulsars at different sky locations.
We will not be discussing the actual data analysis techniques in-depth here since most of them are rather irrevelant. One thing that we will mention here is the *expected cross-correlation* between two pulsars separated by some angular separation due to a common, isotropic (and possibly *stochastic*) GW signal, this is known as the Hellings and Downs curve. By comparing the measured cross-correlation with this expected cross-correlation (i.e. plotting many *pairs of pulsars* on the plot below), we can identify the presence of a common isotropic GW signal.

In [ ]:
plt.figure(dpi=150)
angles = np.linspace(0, 180, num=500)

@np.vectorize
def expected_corr(angle):
    """
    See either http://articles.adsabs.harvard.edu/pdf/1983ApJ...265L..39H or https://arxiv.org/pdf/1412.1142.pdf
    """
    x = 0.5*(1.0 - np.cos(np.deg2rad(angle)))
    return (1./2) - (1./4)*x + (3./2)*x*np.log(x)

plt.xlabel("angle [degree]")
plt.ylabel("expected cross-correlation")
plt.xlim(0, 180)
plt.plot(angles, expected_corr(angles))

## BWDTA: binary white dwarf timing array (a virtual detector)



The gravitational wave emitted by two inspiraling white dwarfs (a.k.a. BWD) is of *mostly constant frequency* in the LISA detection band (~$10^{-4} - 10^{-1}$ Hz). In fact, we can compute, to a very good accuracy, the GW waveform from the BWD system using the quadrupole formula we derived before

$$ h_{+}(t) = \frac{4}{r} \left( \frac{G\mathcal{M}_{c}}{c^2} \right)^{5/3} \left( \frac{\pi f_{\rm GW}}{c} \right)^{2/3} \frac{1 + \cos^2 \theta}{2} \cos\left( 2\pi f_{\rm GW} t_{\rm ret} + 2\phi \right)$$
$$ h_{\times}(t) = \frac{4}{r} \left( \frac{G\mathcal{M}_{c}}{c^2} \right)^{5/3} \left( \frac{\pi f_{\rm GW}}{c} \right)^{2/3} \cos \theta \sin\left( 2\pi f_{\rm GW} t_{\rm ret} + 2\phi \right)$$

where $\mathcal{M}_{c}$ is the chirp mass. That is we model the two white dwarfs as *non-spinning point masses without taking any interaction into account*.

The effect of a propagating VLF GW would be, similar to the PTA case, ***changing the observed GW frequency from a BWD system***. Note that if we are targeting galactic BWDs, then we are safely ignore the cosmological redshift since $z_{\rm BWD} \ll 1$. That is, the VLF GW will be modulating the frequency as (to maximize confusion, we will be using $\nu$ to denote the frequency of the VLF GW)
$$ f^{\rm obs}_{\rm GW} \; = f^{\rm em}_{\rm GW} \times \left[ 1 -  \sum_{{\rm pol} = +,\times} \frac{1}{2} \frac{\hat{p}^i \hat{p}^j}{1 + \hat{p} \cdot \hat{\Omega}} e^{{\rm pol}}_{ij}(\hat{\Omega}) \left( e^{-2\pi i\nu L(1 + \hat{p} \cdot \hat{\Omega})} - 1\right) \tilde{h}_{{\rm pol}}(\nu, \hat{\Omega}) \right]$$

The frequency evolution of the wave due to GW emission is governed by
$$ \dot{f}_{\rm GW} = \frac{96}{5} \pi^{8/3} \left( \frac{G\mathcal{M}_{c}}{c^3} \right)^{5/3} f_{\rm GW}^{11/3} $$

Note that $\frac{GM_{\odot}}{c^3} \approx 4.93 \mu s$, if we put $f_{\rm GW} \approx 10^{-1}$ Hz and $\mathcal{M}_{c} \approx 1.21 M_{\odot}$, and that we observe the signal for 1 week (note: this *does not* have to be the lifetime of the LISA mission, more on this later), the expected change in frequency is

In [ ]:
f_dot = (96./5)*np.pi**(8./3)*(4.93e-6*1.21)*(0.1)**(11./3)
T_obs = 604800 # 1 week in second
delta_f = f_dot * T_obs
print("Expected drift in frequency: {:.3f} Hz".format(delta_f))

This means that roughly
$$ \Delta f^{\rm intrinsic}_{\rm GW} \approx 0.316 \left( \frac{\mathcal{M}_{c}}{1.21 M_{\odot}} \right)^{5/3} \left( \frac{f_{\rm GW}}{0.1 \; \rm {Hz}} \right)^{11/3} \left(\frac{T_{\rm obs}}{1 \; {\rm week}}\right) \; \rm{Hz} $$

Note that we can also solve for the frequency as a function of time from coalescence $\tau$, which is given by
$$ f_{\rm GW} \approx 134 \left( \frac{1.21 M_{\odot}}{\mathcal{M_{c}}} \right)^{5/8} \left( \frac{1 \; {\rm s}}{\tau} \right)^{3/8} \; {\rm Hz}$$

### Sensitivity of BWDTA using the Doppler tracking approach: a *crude* estimate



In [ ]:
tau_enter_LISA_band = (1e-4/134.)**(-8./3.) # Time measured from coalescence that the BWD signal enters the LISA band
tau_leave_LISA_band = (1e-1/134.)**(-8./3.) # Time measured from coalescence that the BWD signal leaves the LISA band
N_obs = 40
times = np.linspace(tau_enter_LISA_band, tau_enter_LISA_band - N_obs*T_obs, num=N_obs) # making 40 observations

@np.vectorize
def f_gw(tau):
    return 134.0*(1./tau)**(3./8.)

@np.vectorize
def h_vlf(t):
    # VLF GW plane wave: h(t) = A*cos(2\pi f t + phi_0)
    A = 1e-7
    f = 1./(10*604800) # Hz
    phi_0 = np.pi/2.
    return A*np.cos(2.*np.pi*f*t + phi_0)

@np.vectorize
def redshift_by_vlf(f_em, t):
    G = 1 # The geometrical pre-factor, set G as 1 for now
    # Account for the Earth term only now
    return f_em - G*(h_vlf(t))

@np.vectorize
def doppler_shift(f_em, v_rel):
    D = np.sqrt((1-v_rel/c)/(1+v_rel/c))
    return D*f_em

plt.figure(dpi=150)
plt.scatter((tau_enter_LISA_band - times[::-1])/T_obs, f_gw(times)[::-1], marker="x", label="VLF GW absent")
plt.scatter((tau_enter_LISA_band - times[::-1])/T_obs, redshift_by_vlf(f_gw(times)[::-1], times[::-1]), marker="o", label="VLF GW present, " + r"$A = 1 \times 10^{-7}, f_{\rm VLF} = 1.65 \times 10^{-7}$" + " Hz", zorder=-10)
plt.ylim(0.998e-4, 1.002e-4)
plt.xlabel("number of observations")
plt.ylabel(r"$f_{\rm GW}$ [Hz]")
plt.legend()
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

Can you see the VLF plane wave? Note that this does not say anything about the amplitude of the VLF GW that we can detect -- one can always change the scale of the y-axis of the plot so that the deviation can be seen by eyes!

Now, we can already place some constraints on the GWs that we can detect using a single BWD system:


1) Assuming that we can measure the GW frequency of a BWD inspiral signal *perfectly*, the change in frequency *due to the VLF GW* must be larger than the shift in frequency due to the BWD intrinsic dynamics. This imposes a limit on *amplitude of the VLF GW* we can detect using this setup


2) since we are trying to detect GWs by observing the change in BWD GW frequency over the LISA mission (about 4 years), the VLF GW plane wave should at least complete one cycle, the lowest possible VLF frequency that we are sensitive to is

In [ ]:
f_VLF_lower_bound = 1/(4*3.154e+7) # Change to your favorite number
print("Lower sensitive frequency: {:.3E} Hz".format(f_VLF_lower_bound))

3) Since we are measuring the BWD GW frequency using data segment of length $T_{\rm obs}$, the `VLF' GW should have a frequency $f_{\rm VLF} \ll \frac{1}{T_{\rm obs}}$

In [ ]:
f_VLF_upper_bound = 1./T_obs
print("Upper sensitive frequency: {:.3E} Hz".format(f_VLF_upper_bound))


Now we can plot the sensitivity (in characteristic strain) of our ideal BWD'TA' (because this is for a single BWD)

In [ ]:
# Plot the BWDTA sensitivity plot

# Simple Phase Modulation and Signal Reconstruction

$$ h_{obs} = h_{BWD}e^{-i2\pi f_0t} \times \left(1+ \frac{i\Gamma}{2}(e^{-i2\pi ft} + e^{i2\pi ft} )\right)$$

$\Gamma \sim h_{SS}$ where $h_{SS}$ is the strain amplitude from a secondary source which is of frequency $f$.

Define the time-domain waveform from a solitary Binary White Dwarf (BWD) with frequency `f0`, strain amplitude `h_BWD` in `carr()`\
Assume a phase modulation caused by a secondary source with frequency `f` and modulation depth `Gamma` in `net_waveform_tf()`

In [ ]:
# Some useful functions
def days_to_secs(t):
    return t*24*3600

def wrap_phase(arr, res=0.04):
    for i in range(len(arr)-1):
        if arr[i+1]-arr[i] >= 2*np.pi-res:
            arr[:i+1] = arr[:i+1] + 2*np.pi
        elif arr[i+1]-arr[i] <= -2*np.pi+res:
            arr[:i+1] = arr[:i+1] - 2*np.pi
        else:
            continue
    return arr

In [ ]:
def carr(t, h_BWD=1, f0=1e-3):
    w0 = 2*np.pi*f0
    return h_BWD*np.exp(-1j*w0*t)

def modulation(t, f=1e-6, Gamma=.1):
    w = 2*np.pi*f
    return (1 + 1j*Gamma/2 *(np.exp(-1j*w*t) + np.exp(1j*w*t)))


In [ ]:
t = np.linspace(1,10, 100)
t_s = days_to_secs(t)

sig_obs = np.abs(carr(t_s)*modulation(t_s))**2
plt.plot(t, sig_obs)

In [ ]:
ang = np.angle(-carr(t_s)*modulation(t_s))

plt.figure(dpi=100)
plt.scatter(t,ang)
plt.xlabel("Time [days]")
plt.ylabel(r"Phase [rad]")
plt.legend()

In [ ]:
carr_mod = np.angle(-carr(t_s))
sig = ang - carr_mod
plt.figure(dpi=100)
plt.scatter(t, sig)
plt.ylabel(r'Unwrapped residual phase of secondary signal [rad]')
plt.xlabel(r'Time [days]');

In [ ]:
plt.figure(dpi=150)
plt.scatter(t, wrap_phase(sig))
plt.ylabel(r'Residual phase of secondary signal [rad]')
plt.xlabel(r'Time [days]');

To obtain the parameters $\Gamma$ and $f$ of the signal, 

### Measurement error: a Fisher-matrix analysis

### MCMC fitting for phase residual

In [ ]:
import emcee


In [ ]:
!pip install emcee